<a href="https://colab.research.google.com/github/shinyabe/-/blob/main/%E5%8D%92%E8%AB%96%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#卒論シミュレーション
import numpy as np  # 数値計算ライブラリ
import pandas as pd  # データフレーム操作用
import itertools  # 全戦略の組み合わせ生成に使用
import random  # 乱数生成

# 定数（シミュレーション日数や消費者需要）※消費者数は正規分布に従う
DAYS = 30  # シミュレーション日数
CONSUMER_MEAN = 100  # 1日の平均来店者数
CONSUMER_STD = 20  # 1日の来店者数の標準偏差
VARIATION_COEFF = CONSUMER_STD / CONSUMER_MEAN  # 消費者数の変動係数

# 食材の劣化パラメータ設定
MU1, MU2 = 5, 2  # 状態1→2および状態2→3の平均時間
S11, S22 = 2, 1  # 状態1→2および状態2→3の時間の分散
RHO = 0.8  # 相関係数
#卸売業者の目利きコスト戦略が食材の平均寿命（t1,t2の平均）と分散に与える影響度合いの設定
ALPHA_X = 1.2  # 目利きコストによる平均寿命の拡張倍率
ALPHA_Y = 0.8  # 目利きコストによる分散縮小倍率
#卸売業者と小売業者の保存方法戦略が、食材の鮮度低下速度に与える影響度合いを設定
BETA_X, BETA_Y = 0.5, 1.0  # 高品質保存と低価格保存の劣化速度

# 消費者が購入する数の最大値（仮に設定）
MAX_PURCHASE_PER_CUSTOMER = 3

# 卸売・小売の戦略の全パターン生成
#卸売業者の戦略
wholesaler_strategies = list(itertools.product(
    ['high', 'low'],  # 安全在庫量（多め or 少なめ）
    ['high', 'low'],  # 目利きコスト（かける or かけない）
    ['high', 'low'],  # 冷蔵保存（高品質 or 低価格）
    ['high', 'low'],  # エチレン除去（未使用）
    ['FIFO', 'LIFO']  # 出荷順（先入先出 or 後入先出）
))

#小売店の戦略
retailer_strategies = list(itertools.product(
    ['high', 'none'],  # 割引戦略（有 or 無）
    ['high', 'low']   # 冷蔵保存戦略
))

# 状態遷移時間の生成（2次元正規分布）
# 状態1→2および状態2→3の平均時間（mu1,mu2）、と分散（s11,s22）、相関係数（rho）を持つ2次元正規分布から食材の劣化に関する2つの時間パラメータt1,t2を生成
def generate_lifetimes(mu1, mu2, s11, s22, rho):
    cov = [[s11, rho * np.sqrt(s11 * s22)], [rho * np.sqrt(s11 * s22), s22]]  # 共分散行列
    return np.random.multivariate_normal([mu1, mu2], cov)  # t1, t2を生成

# 鮮度状態の判定関数（状態1,2,3）
#食材の生産日（product_day）、現在の日付(today)、食材固有の劣化時間(t1,t2)、および鮮度低下速度(fd)を使用して、食材の腐敗進捗度(fp)を算出
def determine_state(produced_day, today, t1, t2, fd):
    fp = -fd * (today - produced_day) + t1 + t2  # 腐敗進捗度
    if fp > t2:
        return 1  # 状態1（良品）
    elif fp > 0:
        return 2  # 状態2（スレ傷あり）
    else:
        return 3  # 状態3（腐敗）

# 消費者が購入するかの判断（状態と値引き）
def consumer_decides(state, discounted):  #食材の状態(state)と小売店が割引を行っているかの有無(discounted)
    if state == 1: #状態1であれば購入
        return True
    elif state == 2 and discounted: #状態2では割引を行なっている場合にのみ購入
        return True
    return False #状態3の食材は購入されない

# 1戦略のシミュレーション実行関数
def run_simulation(wh_strategy, ret_strategy): #卸売業者 (wh_strategy) と小売業者 (ret_strategy) の戦略の組み合わせを受け取る
    total_r_profit = 0  # 小売利益
    total_discard = 0  # 廃棄量
    inventory = []  # 小売在庫

    for day in range(DAYS):
        # 卸の食材納入
        mu1, mu2 = MU1, MU2
        s11, s22 = S11, S22
        fd = BETA_X if wh_strategy[2] == 'high' else BETA_Y  # 卸の保存方式による腐敗速度

        # 目利き戦略が高い場合、平均を延ばし分散を縮小
        if wh_strategy[1] == 'high':
            mu1 *= ALPHA_X
            mu2 *= ALPHA_X
            s11 *= ALPHA_Y
            s22 *= ALPHA_Y

        # 卸売業者の安全在庫量戦略（未使用ですが、将来的な拡張のために残します）
        # if wh_strategy[0] == 'high':
        #     pass # 多めに納入するロジックをここに追加
        # else:
        #     pass # 少なめに納入するロジックをここに追加

        t1, t2 = generate_lifetimes(mu1, mu2, s11, s22, RHO)
        inventory.append({'produced_day': day, 't1': t1, 't2': t2, 'fd': fd}) #小売在庫 (inventory) に追加

        # 消費者数を正規分布からサンプリング
        customer_num = int(np.random.normal(CONSUMER_MEAN, CONSUMER_STD)) #消費者数をランダムに生成
        sold = 0 #販売数の初期値
        discarded = 0 #廃棄量の初期値

        # 小売の戦略設定
        discount = ret_strategy[0] == 'high' # 小売戦略の0番目が割引戦略になりました
        cold_fd = BETA_X if ret_strategy[1] == 'high' else BETA_Y # 小売戦略の1番目が冷蔵保存戦略になりました

        # 出荷順序（FIFO or LIFO）
        inventory = sorted(inventory, key=lambda x: x['produced_day'], reverse=(wh_strategy[4] == 'LIFO'))

        # 購入処理（1人ランダムな数を購入）
        for _ in range(customer_num):
            # 各消費者が購入する数をランダムに決定（1個からMAX_PURCHASE_PER_CUSTOMER個まで）
            items_to_buy = random.randint(1, MAX_PURCHASE_PER_CUSTOMER)
            bought_count = 0
            # 在庫を順番に見て、購入可能な商品をitems_to_buyの数だけ探す
            # 在庫リストをコピーしてイテレーション中に削除できるようにする
            for item in inventory[:]:
                if bought_count < items_to_buy:
                    state = determine_state(item['produced_day'], day, item['t1'], item['t2'], item['fd'] + cold_fd)
                    if consumer_decides(state, discount):
                        sold += 1
                        inventory.remove(item)
                        bought_count += 1
                else:
                    # 必要な数を購入したらこの消費者については次の商品を探すのをやめる
                    break


        # 腐敗品の廃棄処理
        # 在庫リストをコピーしてイテレーション中に削除できるようにする
        for item in inventory[:]:
            state = determine_state(item['produced_day'], day, item['t1'], item['t2'], item['fd'] + cold_fd)
            if state == 3:
                inventory.remove(item)
                discarded += 1

        revenue = sold * (120 if discount else 150)  # 単価設定 (割引あり120, なし150)
        # 在庫コスト：その日の終わりに残っている在庫数 * 1個あたりの保管コスト（例: 10）
        # 廃棄コスト：その日に廃棄された数 * 1個あたりの廃棄にかかるコスト（例: 50）
        cost = len(inventory) * 10 + discarded * 50
        total_r_profit += revenue - cost
        total_discard += discarded

    return total_r_profit, total_discard

# 全組合せでシミュレーション実行
results = []
# 小売戦略のタプル構造が変わったため、ここで調整します
# 以前: (安全在庫量, 割引戦略, 冷蔵保存戦略)
# 修正後: (割引戦略, 冷蔵保存戦略)
# 卸売戦略のタプル構造は変わりません
# (安全在庫量, 目利きコスト, 冷蔵保存, エチレン除去, 出荷順)




for w in wholesaler_strategies:
    for r in retailer_strategies:
        # 小売戦略のタプル構造変更に伴い、run_simulationに渡す際に調整が必要ですが、
        # retailer_strategiesの定義自体を変更したので、そのまま渡せます。
        profit, discard = run_simulation(w, r)
        results.append({
            '卸戦略': w,
            '小売戦略': r,
            '小売利益': profit,
            '廃棄量': discard
        })

# DataFrameで結果出力
df_results = pd.DataFrame(results)
print(df_results)  # 結果表示（利益・廃棄量）

                                卸戦略          小売戦略  小売利益  廃棄量
0    (high, high, high, high, FIFO)  (high, high)  3600    0
1    (high, high, high, high, FIFO)   (high, low)  3600    0
2    (high, high, high, high, FIFO)  (none, high)  4500    0
3    (high, high, high, high, FIFO)   (none, low)  4500    0
4    (high, high, high, high, LIFO)  (high, high)  3600    0
..                              ...           ...   ...  ...
123      (low, low, low, low, FIFO)   (none, low)  4500    0
124      (low, low, low, low, LIFO)  (high, high)  3600    0
125      (low, low, low, low, LIFO)   (high, low)  3600    0
126      (low, low, low, low, LIFO)  (none, high)  4500    0
127      (low, low, low, low, LIFO)   (none, low)  4500    0

[128 rows x 4 columns]


In [1]:
# -*- coding: utf-8 -*-
"""
三分類 × 卸・小売・消費者 の離散時間シミュレーション
- 保存：通常/低温
- 価格：通常/一定率割引
- 劣化：3状態の比例遷移（保存方式で遷移率が低下）
- 出荷順序：FIFO/LIFO（状態2優先 or 状態1優先の簡易近似）
"""

from dataclasses import dataclass, field
from typing import Dict, Literal, Tuple
import numpy as np
import pandas as pd

State = Literal[1, 2, 3]            # 1=良品, 2=傷み, 3=腐敗
Node = Literal["W", "R"]            # W=卸, R=小売
StorageMode = Literal["norm", "cold"]
ShipRule = Literal["FIFO", "LIFO"]

# --------------------------
# パラメタ定義
# --------------------------
@dataclass
class CategoryParam:
    name: str
    # 卸W/小売R × （通常/低温）に応じた 1->2, 2->3 の遷移率スケール
    # 低温は tau で通常より遅くなる（0<tau<1）
    lambda_W_norm: Tuple[float, float]   # (lambda1, lambda2)
    lambda_R_norm: Tuple[float, float]
    tau_W: Tuple[float, float]           # (tau1, tau2) for cold
    tau_R: Tuple[float, float]
    # 価格
    retail_base_price: float             # p^{c,1}
    retail_discount_rate: float          # 0〜1（状態2の一定率割引）
    wholesale_price: float               # a^c
    # 需要ベース & 品質（追熟のピークはここで反映）
    demand_mean: float                   # 平均需要（Poisson/Normalで使用）
    demand_sigma: float = 0.0            # 正規分布で使う標準偏差
    use_poisson: bool = True
    # 追熟用の品質係数曲線（任意）：Noneのときは1.0で一定
    ripening_curve: np.ndarray = None    # 長さTのベクトル（果物のみ）

@dataclass
class CostParam:
    # 保存コスト（在庫1単位×日）
    store_CO2_W_norm: float
    store_CO2_W_cold: float
    store_CO2_R_norm: float
    store_CO2_R_cold: float

    # CO2（輸送・廃棄）
    trans_CO2_per_unit: float
    waste_CO2_per_unit: float

    # 金銭コスト例（必要に応じて0でも可）
    store_cost_W_norm: float = 0.0
    store_cost_W_cold: float = 0.0
    store_cost_R_norm: float = 0.0
    store_cost_R_cold: float = 0.0
    inspect_cost_per_unit: float = 0.0   # 検品でかかった単価×検査対象
    waste_cost_per_unit: float = 0.0     # 廃棄の金銭コスト（小売側）

@dataclass
class InspectionParam:
    on: bool                 # 検品するか
    rho2: float              # 実施時：状態2として除去率
    rho3: float              # 実施時：状態3として除去率
    hat_rho2: float          # 不実施時：自然発見の除去率（通常は小さい）
    hat_rho3: float

@dataclass
class Policy:
    storage_W: StorageMode = "norm"    # 卸の保存モード（norm/cold）
    storage_R: StorageMode = "norm"    # 小売の保存モード
    ship_rule: ShipRule = "FIFO"       # 出荷順序：FIFOは状態2優先, LIFOは状態1優先
    # 卸の入荷（発注到着）を日次で与える関数：A_t^c を返す
    # signature: fn(day:int, cat_name:str) -> float
    arrival_fn: callable = None

# --------------------------
# ユーティリティ
# --------------------------
def clip_nonneg(x):
    return max(0.0, float(x))

def get_lambda(lambda_norm, tau, storage: StorageMode):
    l1, l2 = lambda_norm
    t1, t2 = tau
    if storage == "norm":
        return l1, l2
    else:
        return l1 * t1, l2 * t2

# --------------------------
# メインシミュレータ
# --------------------------
class SupplyChainSim:
    def __init__(self, T: int,
                 categories: Dict[str, CategoryParam],
                 costs: CostParam,
                 inspection: InspectionParam,
                 policy: Policy,
                 seed: int = 42):
        self.T = T
        self.cats = categories
        self.costs = costs
        self.inspection = inspection
        self.policy = policy
        self.rng = np.random.default_rng(seed)

        # 在庫（状態別）
        self.I = {c: {1: 0.0, 2: 0.0, 3: 0.0} for c in self.cats}  # 卸
        self.J = {c: {1: 0.0, 2: 0.0, 3: 0.0} for c in self.cats}  # 小売

        # 記録用
        self.logs = []

    # 需要生成
    def _draw_demand(self, day: int, cp: CategoryParam) -> float:
        base = cp.demand_mean
        if not cp.use_poisson and cp.demand_sigma > 0:
            val = self.rng.normal(base, cp.demand_sigma)
        else:
            val = self.rng.poisson(base)
        # 追熟（ripening_curve）があれば需要倍率として掛ける
        if cp.ripening_curve is not None:
            r = float(cp.ripening_curve[day])
            val = max(0.0, val * r)
        return float(val)

    # 卸の入荷と検品（A_t^c -> 受入 & 廃棄）
    def _wh_inbound_and_inspection(self, A: float, cp: CategoryParam) -> Tuple[float, float, float]:
        if self.inspection.on:
            xi2, xi3 = self.inspection.rho2, self.inspection.rho3
        else:
            xi2, xi3 = self.inspection.hat_rho2, self.inspection.hat_rho3
        xi2 = np.clip(xi2, 0, 1)
        xi3 = np.clip(xi3, 0, 1)
        removed = A * (xi2 + xi3)
        recv = A - removed  # 受け入れは状態1として在庫投入
        return recv, removed, A  # (受入, 検品廃棄, 検査対象数量)

    # 卸の出荷配分（FIFO/LIFO 近似）
    def _wh_ship(self, c: str, demand_from_retail: float, rule: ShipRule) -> Dict[State, float]:
        """小売からの要望数量（総量 demand_from_retail）に対し、状態別で出荷を割付け
           - FIFO: state2優先 → state1
           - LIFO: state1優先 → state2
        """
        avail1, avail2 = self.I[c][1], self.I[c][2]
        ship = {1: 0.0, 2: 0.0, 3: 0.0}
        need = demand_from_retail

        if rule == "FIFO":
            # 傷から先に出す（古いもの優先という近似）
            d2 = min(avail2, need)
            ship[2] = d2
            need -= d2
            d1 = min(avail1, need)
            ship[1] = d1
        else:  # LIFO
            d1 = min(avail1, need)
            ship[1] = d1
            need -= d1
            d2 = min(avail2, need)
            ship[2] = d2

        # 在庫から差し引き
        self.I[c][1] -= ship[1]
        self.I[c][2] -= ship[2]
        return ship

    # 状態遷移（比例）
    def _transition(self, node: Node, c: str):
        cp = self.cats[c]
        if node == "W":
            l1, l2 = get_lambda(cp.lambda_W_norm, cp.tau_W, self.policy.storage_W)
            stock = self.I[c]
        else:
            l1, l2 = get_lambda(cp.lambda_R_norm, cp.tau_R, self.policy.storage_R)
            stock = self.J[c]

        # 遷移
        move12 = l1 * stock[1]
        move23 = l2 * stock[2]
        # clamp
        move12 = min(move12, stock[1])
        move23 = min(move23, stock[2])

        stock[1] -= move12
        stock[2] += move12
        stock[2] -= move23
        stock[3] += move23

    def run(self):
        for t in range(self.T):
            day_log = {
                "t": t,
                "rev_W": 0.0, "rev_R": 0.0,
                "cost_W": 0.0, "cost_R": 0.0,
                "CO2_store": 0.0, "CO2_trans": 0.0, "CO2_waste": 0.0,
                "waste_W": 0.0, "waste_R": 0.0,
                "ship_total": 0.0, "sales_total": 0.0,
                "stock_W": 0.0, "stock_R": 0.0,
                "lost_sales": 0.0,
            }

            # ---- 1) 卸の入荷 & 検品（状態1に受入、検品廃棄はWの廃棄へ）
            for cname, cp in self.cats.items():
                A = float(self.policy.arrival_fn(t, cname)) if self.policy.arrival_fn else 0.0
                recv, removed, inspected_qty = self._wh_inbound_and_inspection(A, cp)
                self.I[cname][1] += recv
                day_log["waste_W"] += removed
                day_log["cost_W"] += inspected_qty * self.costs.inspect_cost_per_unit

            # ---- 2) 卸の状態遷移
            for cname in self.cats:
                self._transition("W", cname)

            # ---- 3) 小売の受注量（簡易：各カテゴリで「目標補充量」を需要平均で見込む）
            req_from_R = {cname: self.cats[cname].demand_mean for cname in self.cats}

            # ---- 4) 卸→小売 出荷（状態別割当）
            ship_by_cat = {}
            for cname, req in req_from_R.items():
                ship = self._wh_ship(cname, req, self.policy.ship_rule)
                ship_by_cat[cname] = ship
                # 卸の売上 & CO2(輸送)
                a = self.cats[cname].wholesale_price
                shipped_units = ship[1] + ship[2]
                day_log["rev_W"] += a * shipped_units
                day_log["CO2_trans"] += shipped_units * self.costs.trans_CO2_per_unit
                day_log["ship_total"] += shipped_units

            # ---- 5) 小売の受入（在庫に加算）
            for cname, ship in ship_by_cat.items():
                self.J[cname][1] += ship[1]
                self.J[cname][2] += ship[2]

            # ---- 6) 小売の状態遷移
            for cname in self.cats:
                self._transition("R", cname)

            # ---- 7) 需要発生 & 販売
            for cname, cp in self.cats.items():
                D = self._draw_demand(t, cp)
                # 価格（状態2は一定率割引）
                p1 = cp.retail_base_price
                p2 = (1.0 - cp.retail_discount_rate) * cp.retail_base_price

                # シンプルな優先販売：まず状態1、残れば状態2
                sell1 = min(self.J[cname][1], D)
                D_left = D - sell1
                sell2 = min(self.J[cname][2], max(0.0, D_left))
                self.J[cname][1] -= sell1
                self.J[cname][2] -= sell2

                # 売上
                day_log["rev_R"] += p1 * sell1 + p2 * sell2
                day_log["sales_total"] += (sell1 + sell2)

                # 需要未充足（KPI用）
                day_log["lost_sales"] += max(0.0, D - (sell1 + sell2))

            # ---- 8) 小売の廃棄（状態3は販売不可：ここで廃棄して0に戻す）
            for cname in self.cats:
                waste_R_today = self.J[cname][3]
                if waste_R_today > 0:
                    day_log["waste_R"] += waste_R_today
                    # CO2とコスト
                    day_log["CO2_waste"] += waste_R_today * self.costs.waste_CO2_per_unit
                    day_log["cost_R"] += waste_R_today * self.costs.waste_cost_per_unit
                    # クリア
                    self.J[cname][3] = 0.0

            # ---- 9) 保存CO2/コスト（在庫量に比例）
            for cname, cp in self.cats.items():
                # 卸
                stockW = sum(self.I[cname].values())
                if self.policy.storage_W == "norm":
                    day_log["CO2_store"] += stockW * self.costs.store_CO2_W_norm
                    day_log["cost_W"]     += stockW * self.costs.store_cost_W_norm
                else:
                    day_log["CO2_store"] += stockW * self.costs.store_CO2_W_cold
                    day_log["cost_W"]     += stockW * self.costs.store_cost_W_cold
                # 小売
                stockR = sum(self.J[cname].values())
                if self.policy.storage_R == "norm":
                    day_log["CO2_store"] += stockR * self.costs.store_CO2_R_norm
                    day_log["cost_R"]     += stockR * self.costs.store_cost_R_norm
                else:
                    day_log["CO2_store"] += stockR * self.costs.store_CO2_R_cold
                    day_log["cost_R"]     += stockR * self.costs.store_cost_R_cold

                day_log["stock_W"] += stockW
                day_log["stock_R"] += stockR

            self.logs.append(day_log)

        # 集計出力
        df = pd.DataFrame(self.logs)
        df["pi_W"] = df["rev_W"] - df["cost_W"]
        df["pi_R"] = df["rev_R"] - df["cost_R"]
        df["profit_total"] = df["pi_W"] + df["pi_R"]
        df["CO2_total"] = df["CO2_store"] + df["CO2_trans"] + df["CO2_waste"]
        return df

# --------------------------
# 使い方（例）
# --------------------------
if __name__ == "__main__":
    T = 60  # 60日シミュレーション

    # 追熟（果物）用の品質/需要倍率：中央付近でピーク
    x = np.linspace(-2.5, 2.5, T)
    ripening = 0.6 + 0.6 * np.exp(-(x**2))  # 0.6〜1.2程度

    categories = {
        "長期": CategoryParam(
            name="長期",
            lambda_W_norm=(0.02, 0.01),   # 遅い劣化
            lambda_R_norm=(0.03, 0.015),
            tau_W=(0.5, 0.5),             # 低温で半減
            tau_R=(0.5, 0.5),
            retail_base_price=180,
            retail_discount_rate=0.25,    # 状態2は25%引き
            wholesale_price=80,
            demand_mean=80,               # 平均需要
            use_poisson=True
        ),
        "短期": CategoryParam(
            name="短期",
            lambda_W_norm=(0.12, 0.20),   # 速い劣化
            lambda_R_norm=(0.16, 0.24),
            tau_W=(0.6, 0.6),
            tau_R=(0.6, 0.6),
            retail_base_price=140,
            retail_discount_rate=0.30,
            wholesale_price=60,
            demand_mean=120,
            use_poisson=True
        ),
        "追熟": CategoryParam(
            name="追熟",
            lambda_W_norm=(0.08, 0.12),
            lambda_R_norm=(0.10, 0.16),
            tau_W=(0.6, 0.6),
            tau_R=(0.6, 0.6),
            retail_base_price=200,
            retail_discount_rate=0.20,
            wholesale_price=90,
            demand_mean=100,
            use_poisson=True,
            ripening_curve=ripening
        )
    }

    costs = CostParam(
        store_CO2_W_norm=0.002, store_CO2_W_cold=0.004,
        store_CO2_R_norm=0.002, store_CO2_R_cold=0.004,
        trans_CO2_per_unit=0.001,
        waste_CO2_per_unit=0.05,
        store_cost_W_norm=0.0, store_cost_W_cold=0.01,
        store_cost_R_norm=0.0, store_cost_R_cold=0.01,
        inspect_cost_per_unit=0.0,
        waste_cost_per_unit=0.0
    )

    inspection = InspectionParam(
        on=True, rho2=0.05, rho3=0.02,
        hat_rho2=0.01, hat_rho3=0.005
    )

    # 卸の入荷（発注到着）ルール：ここでは「需要平均の1.05倍」を毎日到着
    def arrival_fn(day: int, cat: str) -> float:
        return 1.05 * categories[cat].demand_mean

    policy = Policy(
        storage_W="cold",   # 卸は低温
        storage_R="norm",   # 小売は通常
        ship_rule="FIFO",   # FIFO近似（傷→良の順で出荷）
        arrival_fn=arrival_fn
    )

    sim = SupplyChainSim(T, categories, costs, inspection, policy, seed=123)
    result = sim.run()

    # 集計表示
    print(result.tail())
    print("\n--- Summary ---")
    print(result[["profit_total", "CO2_total", "waste_R", "waste_W", "lost_sales"]].sum())


     t    rev_W         rev_R  cost_W  cost_R  CO2_store  CO2_trans  \
55  55  22068.9  41988.135231     0.0     0.0   0.641459    0.29295   
56  56  22068.9  40386.687782     0.0     0.0   0.677112    0.29295   
57  57  22068.9  39407.883910     0.0     0.0   0.721975    0.29295   
58  58  22068.9  41076.040479     0.0     0.0   0.747260    0.29295   
59  59  22068.9  42662.763557     0.0     0.0   0.742555    0.29295   

    CO2_waste  waste_W    waste_R  ship_total  sales_total  stock_W  \
55   1.223972    22.05  24.479444      292.95   257.622848      0.0   
56   1.262147    22.05  25.242948      292.95   249.880283      0.0   
57   1.316630    22.05  26.332595      292.95   244.186263      0.0   
58   1.376491    22.05  27.529820      292.95   252.777434      0.0   
59   1.449510    22.05  28.990206      292.95   266.312352      0.0   

       stock_R  lost_sales     pi_W          pi_R  profit_total  CO2_total  
55  320.729385   20.206972  22068.9  41988.135231  64057.035231   2.1